In [48]:
import logging
import math
import operator
from collections import Counter
from pprint import pprint

import numpy as np
import scipy.stats as stats
from pySim.euicc import CardApplicationISDR, GetEuiccChallenge
from pySim.transport.pcsc import PcscSimLink
from statsmodels.sandbox.stats.runs import runstest_1samp
from tqdm.notebook import tqdm

from resimulate.card import Card
from resimulate.util.enums import ISDR_AID

logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

received_challenges: list[str] = []

with PcscSimLink(apdu_tracer=None) as link:
    card = Card(link)
    initialized_card = card.init_card(target_isd_r=ISDR_AID.DEFAULT)
    isd_r = card.runtime_state.mf.applications.get(ISDR_AID.DEFAULT.value.lower(), None)

    if not isd_r:
        print("ISDR not found on card")
        raise SystemExit(1)

    card.runtime_state.lchan[0].select_file(isd_r)
    card.runtime_state.identity["EID"] = CardApplicationISDR.get_eid(
        card.sim_card_commands
    )
    for _ in tqdm(range(50000)):
        challenge = CardApplicationISDR.store_data_tlv(
            card.sim_card_commands, GetEuiccChallenge(), GetEuiccChallenge
        ).to_dict()["get_euicc_challenge"][0]["euicc_challenge"]
        received_challenges.append(challenge)


def entropy(data):
    """Calculate Shannon entropy for byte sequences"""
    counts = Counter(data)
    total = len(data)
    return -sum((count / total) * math.log2(count / total) for count in counts.values())


def test_randomness(byte_strings) -> tuple[np.float64, np.float64, np.float64, float]:
    # Convert hex/base64/raw strings to bytes
    byte_arrays = [
        bytes.fromhex(s)
        if all(c in "0123456789abcdefABCDEF" for c in s)
        else s.encode()
        for s in byte_strings
    ]

    # Flatten to a single byte sequence
    all_bytes = np.concatenate([np.frombuffer(b, dtype=np.uint8) for b in byte_arrays])

    # Chi-Square Test for Byte Uniformity
    byte_counts = np.bincount(
        all_bytes, minlength=256
    )  # Count occurrences of each byte (0-255)
    expected = np.full_like(byte_counts, np.mean(byte_counts))
    chi2_stat, p_chi2 = stats.chisquare(byte_counts, expected)

    # Shannon Entropy
    byte_counts = np.bincount(all_bytes, minlength=256)
    probabilities = byte_counts / all_bytes.size
    entropy_value = stats.entropy(probabilities, base=2)  # entropy(all_bytes)

    # Monobit Test (Balance of 0s and 1s)
    bit_string = "".join(format(byte, "08b") for byte in all_bytes)
    ones = bit_string.count("1")
    zeros = bit_string.count("0")
    monobit_p = stats.binomtest(ones, len(bit_string), p=0.5)

    # Runs Test on Byte Sequences
    z_stat, p_runs = runstest_1samp(all_bytes)

    return (
        round(p_chi2, 6),
        round(entropy_value, 6),
        round(monobit_p.pvalue, 6),
        round(p_runs, 6),
    )


def within_margin(value, target, margin, op: operator) -> bool:
    lower_bound = target - margin
    upper_bound = target + margin
    return op(value, lower_bound) or op(value, upper_bound)


p_chi2, entropy_value, pvalue, p_runs = test_randomness(received_challenges)

if within_margin(p_chi2, 0.05, 0.01, operator.lt):
    print(f"Chi-Square Test: Non-uniform distribution of bytes! ({p_chi2})")
if entropy_value < 8:
    print(f"Shannon Entropy: Low randomness! ({entropy_value})")
if within_margin(pvalue, 0.05, 0.01, operator.lt):
    print(f"Monobit Test: Bias in bits! ({pvalue})")
if within_margin(p_runs, 0.05, 0.01, operator.lt):
    print(f"Runs Test: Sequential patterns! ({p_runs})")

pprint(
    {
        "Chi-Square Test p-value": str(p_chi2),  # <0.05 means not uniform/random
        "Shannon Entropy": str(entropy_value),  # ~8 means high randomness
        "Monobit Test p-value": str(pvalue),  # <0.05 means bias in bits
        "Runs Test p-value": str(p_runs),  # <0.05 means sequential patterns
    }
)


ISDR not found on card


SystemExit: 1

/home/niklas/Documents/documents/uni/master_thesis/resimulate/.venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
